In [1]:
from zipfile import ZipFile
import pandas as pd
import numpy as np

# Загрузка данных

In [2]:
def gen_parcer(f):
    for line in f.readlines():
        yield line.decode().strip().split("\t")

In [3]:
with ZipFile('data2.zip') as datazip:
    with datazip.open('train.tsv') as f:
        pd_train = pd.DataFrame(gen_parcer(f),
                                columns=["context_id", "context_2", "context_1", "context_0", "reply_id", "reply",
                                         "label", "confidence"])
    with datazip.open('final.tsv') as f:
         pd_public = pd.DataFrame(gen_parcer(f),
                                  columns=["context_id", "context_2", "context_1", "context_0", "reply_id", "reply"])

In [4]:
pd_train.head()

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.8753516175
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.9009682113
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.8843202145
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.9825304673
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.8380535096


In [5]:
pd_public.head()

,context_id,context_2,context_1,context_0,reply_id,reply
0,4909294510,,,нет . . . у тебя на лице написано - нет .,0,тогда я попытался с двумя другими женщинами и ...
1,4909294510,,,нет . . . у тебя на лице написано - нет .,1,"я улыбаюсь потому . . . потому что , описывая ..."
2,4909294510,,,нет . . . у тебя на лице написано - нет .,2,"это , так сказать , соответствует уровню моей ..."
3,4909294510,,,нет . . . у тебя на лице написано - нет .,3,я врач .
4,4909294510,,,нет . . . у тебя на лице написано - нет .,4,не обращайте на меня внимания !


In [6]:
pd_train.count()

context_id    97533
context_2     97533
context_1     97533
context_0     97533
reply_id      97533
reply         97533
label         97533
confidence    97533
dtype: int64

In [7]:
pd_public.count()

context_id    104834
context_2     104834
context_1     104834
context_0     104834
reply_id      104834
reply         104834
dtype: int64

In [8]:
target_encoder = {'bad': 0, 'good': 2, 'neutral': 1}  
pd_train.label = pd_train.label.map(target_encoder)

In [9]:
pd_data = pd.concat([pd_train, pd_public], axis=0).reset_index(drop=True)
pd_data.label.fillna(-1, inplace=True)
pd_data.confidence.fillna(0.0, inplace=True)

# Что за данные

In [10]:
pd_train.shape, pd_public.shape, pd_data.shape

((97533, 8), (104834, 6), (202367, 8))

In [11]:
pd_data.groupby("label")["context_id"].count() / len(pd_data)

label
-1.0    0.518039
 0.0    0.171817
 1.0    0.055612
 2.0    0.254533
Name: context_id, dtype: float64

# Простой бейзлайн

In [12]:
import pickle 

In [13]:
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [14]:
import re

In [15]:
w2v_fpath = "w2v/all.norm-sz100-w10-cb0-it1-min100.w2v"
w2v = gensim.models.KeyedVectors.load_word2vec_format(w2v_fpath, binary=True, unicode_errors='ignore')
w2v.init_sims(replace=True)

In [16]:
def clean_text(x):
    x = x.lower()
    x = x.replace("ё", "е")
    return x

In [26]:
empty_vector = np.zeros(100)
def condext_to_vec(x):
    words = re.findall(r"\b(\w+)\b", clean_text(x))
    vectors = sum([w2v[word] for word in words if word in w2v])
    return (empty_vector + vectors).reshape(1, -1)

## features

In [18]:
def to_features_w2v_cross(df):
    s_words = df[["context_2", "context_1", "context_0"]].apply(lambda x: " ".join(x), axis=1)
    context = np.concatenate(s_words.apply(condext_to_vec).tolist())
    
    s_words = df["reply_x"]
    reply_x = np.concatenate(s_words.apply(condext_to_vec).tolist())
    
    s_words = df["reply_y"]
    reply_y = np.concatenate(s_words.apply(condext_to_vec).tolist())
    
    
    features = np.concatenate([context, reply_x, reply_y], axis=1)
    return features

In [19]:
def cross_xy(df):
    pd_cross = pd.merge(df[["context_id", "reply_id", "context_2", "context_1", "context_0", "reply", "label"]],
                        df[["context_id", "reply", "label"]],
                        on="context_id")
    pd_cross["target"] = (pd_cross["label_x"] - pd_cross["label_y"] > 0).astype(int)
    return pd_cross, to_features_w2v_cross(pd_cross), pd_cross["target"].values

## crossval

In [20]:
from tqdm import tqdm_notebook

In [21]:
from sklearn.model_selection import StratifiedKFold

In [22]:
skf = StratifiedKFold(n_splits=3)

In [23]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor

In [24]:
def calc_ndcg_score(df, y_predict):
    df = df.copy()
    df["predict"] = y_predict
    #dcg
    df.sort_values(["context_id", "predict"], ascending=[True, False], inplace=True)
    df["row_count"] = df.groupby("context_id").cumcount()+1
    df["dcg"] = df["label"] / np.log2(df["row_count"] + 1)
    #idcg
    df.sort_values(["context_id", "label"], ascending=[True, False], inplace=True)
    df["row_count"] = df.groupby("context_id").cumcount()+1
    df["idcg"] = df["label"] / np.log2(df["row_count"] + 1)
    
    df_context = df.groupby("context_id").sum()
    return (df_context["dcg"] / df_context["idcg"]).mean() * 100000

In [55]:
def do_cross_val(model_reg, model_fit_params=None):
    ndcg_scores = []

    for _train, _test in tqdm_notebook(skf.split(pd_train.index, pd_train.label), total=3):
        i_train = pd_train.iloc[_train]
        i_test = pd_train.iloc[_test]

        cross_train, X_train, Y_train = cross_xy(i_train)
        cross_test, X_test, Y_test = cross_xy(i_test)

        model_reg.load_weights("model_nn.w")
        model_reg.fit(X_train, Y_train, **model_fit_params)
        Y_predict = model_reg.predict_proba(X_test)
        
        cross_test["prediction"] = Y_predict
        i_test = pd.merge(i_test,
                 cross_test.groupby(["context_id", "reply_id"], as_index=False).prediction.mean(),
                 on=["context_id", "reply_id"])
        ndcg_scores.append(calc_ndcg_score(i_test, i_test.prediction))
        print(ndcg_scores)

    print(ndcg_scores)

## Нормальные варианты

In [37]:
from sklearn.linear_model import LogisticRegression

In [38]:
do_cross_val(LogisticRegression(), {})

A Jupyter Widget


[86229.40078461912, 85854.88226526047, 86156.83494251016]


In [39]:
import lightgbm

In [41]:
do_cross_val(lightgbm.LGBMClassifier(), {})

A Jupyter Widget


[86194.32185744558, 85878.27646359202, 86072.53302072204]


# Сеточка

In [42]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [52]:
model = Sequential([
    Dense(1000, input_dim=300),
    Activation('relu'),
    Dropout(0.5),
    Dense(200),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid'),
])

model.compile(optimizer='adam',
              loss='binary_crossentropy')
model.save_weights("model_nn.w")

In [ ]:
do_cross_val(model,
            {"epochs": 20, "batch_size": 256, "verbose": 2, "validation_split": 0.1})

A Jupyter Widget

Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\ProgramData\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



Train on 345507 samples, validate on 38390 samples
Epoch 1/20
 - 50s - loss: 0.5108 - val_loss: 0.4973
Epoch 2/20
 - 49s - loss: 0.4952 - val_loss: 0.4867
Epoch 3/20
 - 50s - loss: 0.4858 - val_loss: 0.4773
Epoch 4/20
 - 49s - loss: 0.4794 - val_loss: 0.4736
Epoch 5/20
 - 49s - loss: 0.4753 - val_loss: 0.4707
Epoch 6/20
 - 50s - loss: 0.4716 - val_loss: 0.4683
Epoch 7/20
 - 48s - loss: 0.4692 - val_loss: 0.4714
Epoch 8/20
 - 49s - loss: 0.4663 - val_loss: 0.4656
Epoch 9/20
 - 49s - loss: 0.4641 - val_loss: 0.4658
Epoch 10/20
 - 49s - loss: 0.4618 - val_loss: 0.4663
Epoch 11/20
 - 49s - loss: 0.4592 - val_loss: 0.4656
Epoch 12/20
 - 50s - loss: 0.4570 - val_loss: 0.4670
Epoch 13/20
 - 51s - loss: 0.4551 - val_loss: 0.4708
Epoch 14/20
 - 60s - loss: 0.4530 - val_loss: 0.4678
Epoch 15/20
 - 55s - loss: 0.4505 - val_loss: 0.4678
Epoch 16/20
 - 56s - loss: 0.4486 - val_loss: 0.4709
Epoch 17/20
 - 53s - loss: 0.4459 - val_loss: 0.4731
Epoch 18/20
 - 54s - loss: 0.4429 - val_loss: 0.4741
Epoc

# Отладка

In [37]:
_train, _test = next(skf.split(pd_train.index, pd_train.label))
i_train = pd_train.iloc[_train]
i_test = pd_train.iloc[_test]

In [39]:
i_train

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
32457,93647138840824,все не так уж плохо .,сделайте себя полезными .,и это вытянем .,1,будет трудно .,2,0.909092886
32458,93647138840824,все не так уж плохо .,сделайте себя полезными .,и это вытянем .,2,"высоко поднять мы не сможем , так что не медли .",2,0.8059265579
32459,93683303278579,хорошо .,может будет подмога .,капитан лэнс ?,0,"да , мисс гибсон .",2,0.9899184337
32460,93683303278579,хорошо .,может будет подмога .,капитан лэнс ?,1,отставной капитан флота .,2,0.8011402464
32461,93683303278579,хорошо .,может будет подмога .,капитан лэнс ?,2,капитан риггз .,2,0.8468712695
32462,93683303278579,хорошо .,может будет подмога .,капитан лэнс ?,3,капитан башир .,2,0.9221820127
32463,93683303278579,хорошо .,может будет подмога .,капитан лэнс ?,4,"нет , капитан дженкинс .",2,0.9927068992
32464,93683303278579,хорошо .,может будет подмога .,капитан лэнс ?,5,капитан даффи .,2,0.9389570234
32465,93695962253917,вперед . привет .,"у нас квест и первое задание , повесить его по...","а кто он , я его не знаю .",0,"я не знаю , как его зовут .",2,0.9389397554
32466,93695962253917,вперед . привет .,"у нас квест и первое задание , повесить его по...","а кто он , я его не знаю .",1,"царь иудеи , некто христос .",2,0.7832186689


In [103]:
X_train = to_features_w2v(i_train)
X_test = to_features_w2v(i_test)
Y_train = trg_label_confidence(i_train)
Y_test = trg_label_confidence(i_test)

In [109]:
model.fit(X_train, Y_train, epochs=10, batch_size=256, verbose=2, validation_split=0.2)

Train on 52016 samples, validate on 13005 samples
Epoch 1/10
 - 16s - loss: 0.5819 - val_loss: 0.6090
Epoch 2/10
 - 16s - loss: 0.5776 - val_loss: 0.6067
Epoch 3/10
 - 16s - loss: 0.5743 - val_loss: 0.6061
Epoch 4/10
 - 16s - loss: 0.5680 - val_loss: 0.6102
Epoch 5/10
 - 16s - loss: 0.5661 - val_loss: 0.6084
Epoch 6/10
 - 16s - loss: 0.5601 - val_loss: 0.6073
Epoch 7/10
 - 16s - loss: 0.5571 - val_loss: 0.6085
Epoch 8/10
 - 17s - loss: 0.5527 - val_loss: 0.6110
Epoch 9/10
 - 17s - loss: 0.5476 - val_loss: 0.6043
Epoch 10/10
 - 16s - loss: 0.5434 - val_loss: 0.6089


In [110]:
Y_predict = model.predict(X_test)

In [111]:
print(mean_absolute_error(Y_test, Y_predict))
print(calc_ndcg_score(i_test, Y_predict))

0.69282322577
86568.10480536966


In [113]:
model.load_weights("model_nn.w")

TypeError: 'list' object is not callable

# Submit

In [119]:
def do_submit(submission_name, model_reg, to_features, to_target, model_fit_params=None):
    i_train = pd_train
    i_test = pd_public.copy()

    X_train = to_features(i_train)
    X_test = to_features(i_test)
    Y_train = to_target(i_train)

    model_reg.fit(X_train, Y_train, **model_fit_params)
    Y_predict = model_reg.predict(X_test)

    i_test["predict"] = Y_predict
    i_test.sort_values(["context_id", "predict"], ascending=[True, False], inplace=True)

    i_test[["context_id", "reply_id"]].to_csv(submission_name, index=False, header=False, sep="\t")
    
    return i_test

In [120]:
do_submit("w2v_nn_mlp.tsv", model, to_features_w2v, trg_label_confidence,
          {"epochs": 10, "batch_size": 256, "verbose": 2, "validation_split": 0.1}).head(15)

Train on 87779 samples, validate on 9754 samples
Epoch 1/10
 - 28s - loss: 0.6091 - val_loss: 0.6085
Epoch 2/10
 - 26s - loss: 0.6053 - val_loss: 0.6071
Epoch 3/10
 - 26s - loss: 0.6017 - val_loss: 0.6035
Epoch 4/10
 - 26s - loss: 0.5986 - val_loss: 0.6038
Epoch 5/10
 - 26s - loss: 0.5942 - val_loss: 0.6013
Epoch 6/10
 - 27s - loss: 0.5901 - val_loss: 0.6014
Epoch 7/10
 - 27s - loss: 0.5877 - val_loss: 0.5985
Epoch 8/10
 - 27s - loss: 0.5846 - val_loss: 0.5995
Epoch 9/10
 - 26s - loss: 0.5820 - val_loss: 0.5971
Epoch 10/10
 - 28s - loss: 0.5782 - val_loss: 0.6016


,context_id,context_2,context_1,context_0,reply_id,reply,predict
3479,100097508986637,,,выключай их,4,"пытаюсь , сэр .",0.640513
3478,100097508986637,,,выключай их,3,я пытаюсь !,0.464230
3476,100097508986637,,,выключай их,1,"ты не выключишь , тогда € сам выключу",0.265819
3475,100097508986637,,,выключай их,0,выключить что ?,0.057735
3480,100097508986637,,,выключай их,5,выключите,-0.237809
3477,100097508986637,,,выключай их,2,выключить ?,-0.328770
3486,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,5,"что "" не надо "" ? не надо меня .",0.478471
3484,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,3,мне уже легче .,0.422974
3483,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,2,я не могу позволить вам уйти .,0.385519
3481,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,0,ладно .,0.176011
